In [1]:
# Importando as bibliotecas
import bs4 
from bs4 import BeautifulSoup
import urllib.request as urllib_request
from urllib.request import Request, urlopen, urlretrieve
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import pandas as pd

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pd.__version__)

BeautifulSoup -> 4.9.3
urllib -> 3.8
pandas -> 1.2.4


In [2]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(400)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/casas/?pagina=' + str(i) + '&transacao=Venda&tipoUnidade=Residencial,Casa&tipo=Im%C3%B3vel%20usado'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price'})
        if valor == None:
            card['valor'] = str('Vazio')
        else :
            card['valor'] = str(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))
 

        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = str('Não disponivel', 'Não disponivel')
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quartos'] = str(0)
        else :
            card['quartos'] = str(quartos.get_text().replace(' ','').replace('\n',''))
        
        
        #vagas
        garagem = anuncio.find('li',{'class':'feature__item text-small js-parking-spaces'})
        if garagem == None:
            card['garagem'] = str(0)
        else :
            card['garagem'] = str(garagem.get_text().replace(' ','').replace('\n',''))
        
            
        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = str(0)
        else :
            card['banheiro'] = str(banheiro.get_text().replace(' ','').replace('\n',''))
        
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)'] = str(0)
        else :
            card['area(m2)'] = str(metro.get_text().replace(' ','').replace('\n','')).replace('m²','')
        
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = tipo.get_text()
        
        cards.append(card)

        
dataset = pd.DataFrame(cards)
dataset.to_csv('./dados/dataset-casas-geral.csv',sep=';', index=False,encoding='utf-8-sig')
dataset

,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),novidade
0,650000,"Casa localizada na Rua Augusto Jung, bairro Ce...",Rua Augusto Jung,Centro,4,4,3,213,Super destaque
1,189000,A casa está localizada no bairro que mais cres...,Rua Assis Gomes,Guagiru,3,2,2,75,Super destaque
2,11900000,Casa finamente decorada com 2 quartos e 1 suít...,Rua Professor Antônio D'Ávila,Setimo Ceu,3,8,3,926,Super destaque
3,345000,"Casa nova no bairro Residencial Santa Paula, r...",Residencial Santa Paula,Jacareí,3,2,2,133,Nenhuma
4,330000,"Casa no Jardim Santa Julia, com 2 dormitórios,...",Jardim Santa Júlia,São José dos Campos,2,5,2,72,Nenhuma
...,...,...,...,...,...,...,...,...,...
9595,1596000,Excelente sobrado contendo 3 suítes com armári...,Vila Mazzei,São Paulo,4,0,6,280,Nenhuma
9596,750000,Sobrado Jd. Vila Galvão Guarulhos Mobiliado 4 ...,Jardim Vila Galvão,Guarulhos,4,2,3,225,Nenhuma
9597,583000,"Casa 132 m² com 2 dormitórios, localizado no B...",Rua dos Crisântemos,Balneario Florida,2,1,2,132,Nenhuma
9598,745000,Sobrado 156 m² com 2 dormitórios sendo 1 suíte...,Rua Engenheiro Isac Garcez,Vila Caminho do Mar,2,2,2,156,Nenhuma
